# Proyecto Mi Bici

In [ ]:
# Importación de librerías
import pandas as pd
import numpy as np
import plotly as plt
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import warnings
import plotly.express as px
warnings.filterwarnings('ignore')
from geopy.distance import geodesic

In [ ]:
# Creación del dataset mediante una unión de diversos archivos 
data= pd.DataFrame()
# Lista de meses disponibles
months =["01","02","03", "04","05","06","07", "08", "09", "10"]
for i in months:
    path= "datos_base/datos_abiertos_2022_"
    t_data=pd.read_csv(path + i +".csv", encoding='latin-1')
    data= data.append(t_data)

In [ ]:
# Lectura del dataset nomenclatura, que representa las estaciones de MiBici
path = "datos_base/"
nomen = pd.read_csv(path + "nomenclatura_2022_10.csv", encoding='latin-1')

In [ ]:
# Datasets de estaciones MiBici
nomen

In [ ]:
# Dataset principal
data.head(5)
data2 = data
data

In [ ]:
# Primero 5 registros de las estaciones de MiBici
nomen.head(5)

In [ ]:
# Detalles del dataset de viajes
data.info()

In [ ]:
# Detalles del dataset de estaciones MiBici
nomen.info()

In [ ]:
# Conteo de datos nules en el dataset de viajes
data.isnull().sum()

In [ ]:
# Forma del dataset principal
data.shape

In [ ]:
# Estadística descriptiva del dataset de viajes
data.describe().T

In [ ]:
# Obtención de diferencia del tiempo de viaje & transformación de datos a tipo fecha
data['Fin_del_viaje']= pd.to_datetime(data['Fin_del_viaje'])
data['Inicio_del_viaje']=pd.to_datetime(data['Inicio_del_viaje'])
data['total_min'] = (data['Fin_del_viaje'] - data['Inicio_del_viaje'])
data['total_min'] = data['total_min'].astype('timedelta64[m]')

In [ ]:
# Unión de los datos para detalle de estación de inicio
data = pd.merge(data, nomen[['id', 'name', 'obcn', 'location', 'longitude','latitude']], how='left',left_on=['Origen_Id'],right_on=['id'])
data.columns =['Viaje_Id', 'Usuario_Id', 'Genero', 'Año_de_nacimiento','Inicio_del_viaje', 'Fin_del_viaje', 'Origen_Id', 'Destino_Id',
       'total_min', 'id_origen', 'name_origen', 'obcn_origen', 'location_origen', 'longitude_origen', 'latitude_origen']
# Unión de los datos para detalle de estación de destino
data = pd.merge(data, nomen[['id', 'name', 'obcn', 'location', 'longitude','latitude']], how='left',left_on=['Destino_Id'],right_on=['id'])
data.columns =['Viaje_Id', 'Usuario_Id', 'Genero', 'Año_de_nacimiento','Inicio_del_viaje', 'Fin_del_viaje', 'Origen_Id', 'Destino_Id',
       'total_min', 'id_origen', 'name_origen', 'obcn_origen', 'location_origen', 'longitude_origen', 'latitude_origen', 
               'id_destino', 'name_destino', 'obcn_destino', 'location_destino', 'longitude_destino', 'latitude_destino']

In [ ]:
# Resultados de la unión
data

In [ ]:
# Primeros dos Registros tras la unión
data.head(2)

In [ ]:
# Transformación sugerida
# def geodesic_calc(lat1, lng1, lat2, lng2):
#    return geodesic((lat1,lng1), (lat2,lng2)).kilometers

In [ ]:
#data['distancia'] = data.apply(lambda row : geodesic_calc(row['latitude_origen'],row['longitude_origen'], row['latitude_destino'], row['longitude_destino']), axis = 1)

In [ ]:
# Inicialización de la característica distancia en 0
data['distancia']=0
# Obtención de la distancia geodésica, representada en kilómetros 
for i in range(len(data['distancia'])):
    lat1 = data['latitude_origen'][i]
    lon1 = data['longitude_origen'][i]

    lat2 = data['latitude_destino'][i]
    lon2 = data['longitude_destino'][i]

    data['distancia'][i]= geodesic((lat1,lon1), (lat2,lon2)).kilometers

In [ ]:
data.total_min[6]

In [ ]:
# Exportar el dataset con distancias y tiempos
data.to_csv('Data.csv',encoding='UTF-8')
print("Success")

In [ ]:
# Graficación de las estaciones de MiBici
fig_map = px.scatter_mapbox(nomen, lat="latitude", 
                            lon="longitude", mapbox_style="stamen-toner", 
                            color_discrete_sequence=["red"], zoom=11, 
                            center = {"lat": 20.676820, "lon": -103.3418228},opacity=1)
fig_map.show()

In [ ]:
# Graficar la frecuencia del tiempo de uso en minutos
data['total_min'].hist(bins=20, color='red',align='mid')
plt.title('Histograma de Tiempo de uso')
plt.xlabel('minutos')
plt.ylabel('Frecuencia')

plt.show()

In [ ]:
# Generar una tabla con rangos de usos basados en el total de minutos de uso
r11=data
condition = (r11['total_min'] >1) & (r11['total_min'] <=10)
r11.loc[condition, ['range min']] = "0-10"
condition = (r11['total_min'] >10) & (r11['total_min'] <=20)
r11.loc[condition, ['range min']] = "20-40"
condition = (r11['total_min'] >20) & (r11['total_min'] <=30)
r11.loc[condition, ['range min']] = "20-30"
condition = (r11['total_min'] >30) & (r11['total_min'] <=40)
r11.loc[condition, ['range min']] = "30-40"
condition = (r11['total_min'] >40) & (r11['total_min'] <=50)
r11.loc[condition, ['range min']] = "40-50"
condition = (r11['total_min'] >50) & (r11['total_min'] <=60)
r11.loc[condition, ['range min']] = "40-50"
condition = (r11['total_min'] >60) & (r11['total_min'] <=120)
r11.loc[condition, ['range min']] = "60-120"
condition = (r11['total_min'] >120)
r11.loc[condition, ['range min']] = ">120"

r11= pd.pivot_table(data, values = 'total_min', index = ['range min'], aggfunc ='count').fillna(0)


In [ ]:
# Resultado de los rangos
r11

In [ ]:
# 
fig = px.bar(r11, x= r11.index, y='total_min')
fig.show()

In [ ]:
# Obtener las columnas de tipo numérico
# para graficarlas mediante box plots
num_columns = data._get_numeric_data().columns
plt.figure(figsize=(10,20))
for i,col in enumerate(num_columns,1):
     plt.subplot(14,1,i)
     sns.boxplot(data[col])
     plt.ylabel(col)
plt.show()

In [ ]:
# Obtener el nombre del día 
data['dia'] = data['Inicio_del_viaje'].dt.weekday
data['nomdia'] = 0
days = ['Lunes', 'Martes', 'Miercoles', 'Jueves', 'Viernes', 'Sabado', 'Domingo']
for i in range(len(data['dia'])):
    data['nomdia'][i] = days[data['dia'][i]]

In [ ]:
# Graficar frecuencia de uso de MiBici por día de la semana
fig = px.pie(data, values='dia', names='nomdia', title='Días más frecuentes')
fig.show()

In [ ]:
# Verificación de la nueva estructura de datos
data.head(2)

In [ ]:
# Nuevas Columnas
data.columns

In [ ]:
# Pivote 20 origen mas usados
data_origen = pd.pivot_table(data, values = 'obcn_origen', index = ['id_origen'], aggfunc ='count').fillna(0)
data_origen = data_origen.sort_values(by=['obcn_origen'], ascending=False)
data_origen = data_origen.reset_index()
data_origen = data_origen.loc[:15]
fig = px.pie(data_origen, values='obcn_origen', names='id_origen', title='Origen más frecuente')
fig.show()
data_origen

In [ ]:
# Pivote 20 destinos mas usados
data_destino = pd.pivot_table(data, values = 'obcn_destino', index = ['id_destino'], aggfunc ='count').fillna(0)
data_destino =data_destino.sort_values(by=['obcn_destino'], ascending=False)
data_destino = data_destino.reset_index()
data_destino =data_destino.loc[:15]
fig = px.pie(data_destino, values='obcn_destino', names='id_destino', title='Destino más frecuente')
fig.show()
data_destino

In [1]:
#fig_map = px.scatter_mapbox(data_origen, lat="latitude", lon="longitude", mapbox_style="stamen-toner", color_discrete_sequence=["red"],
 #                           zoom=11, center = {"lat": 20.676820, "lon": -103.3418228},opacity=1)
#fig_map.show()

In [ ]:
#Histograma de distancia
data['distancia'].hist(bins=20, color='red',align='mid', grid=False)
plt.title('distancia')
plt.show()

In [ ]:
#mapa de top 15 estaciones de origen
data_origen['size'] = 0
for i in range(len(data_origen)):
    data_origen['size'][i] = i


fig_map = px.scatter_mapbox(data_origen, lat="latitude", lon="longitude", mapbox_style="stamen-toner", color_discrete_sequence=["green"],
                            zoom=11, center = {"lat": 20.676820, "lon": -103.3418228},opacity=1, size="size")
fig_map.show()

In [ ]:
#mapa de top 15 estaciones de destino
data_destino['size'] = 0
for i in range(len(data_destino)):
    data_destino['size'][i] = i

fig_map = px.scatter_mapbox(data_destino, lat="latitude", lon="longitude", mapbox_style="stamen-toner", color_discrete_sequence=["blue"],
                            zoom=11, center = {"lat": 20.676820, "lon": -103.3418228},opacity=1, size="size")
fig_map.show()